# DROWSINESS DETECTION MODEL

Here, I shall try to build and train a model from scratch which would be able to detect drowsiness. The model shall be trained on the **MRL Eye Dataset**. This dataset contains infrared images in low and high resolution, all captured in various lightning conditions and by different devices.  

The dataset can be downloaded from [here](http://mrl.cs.vsb.cz/eyedataset "MRL Eye Dataset").  

In the dataset, we annotated the following properties (the properties are indicated in the following order):
- subject ID --> in the dataset, we collected the data of 37 different persons (33 men and 4 women)
- image ID --> the dataset consists of **84,898 images**
- gender [0 - man, 1 - woman] --> the dataset contains the information about gender for each image (man, woman)
- glasses [0 - no, 1 - yes] --> the information if the eye image contains glasses is also provided for each image (with and without the glasses)
- eye state [0 - closed, 1 - open] --> this property contains the information about two eye states (open, close)
- reflections [0 - none, 1 - small, 2 - big] --> we annotated three reflection states based on the size of reflections (none, small, and big reflections)
- lighting conditions [0 - bad, 1 - good] --> each image has two states (bad, good) based on the amount of light during capturing the videos
- sensor ID [01 - RealSense, 02 - IDS, 03 - Aptina] --> at this moment, the dataset contains the images captured by three different sensors (Intel RealSense RS 300 sensor with 640 x 480 resolution, IDS Imaging sensor with 1280 x 1024 resolution, and Aptina sensor with 752 x 480 resolution)

A sample file name, thus, would be *subject ID-image ID-gender-glasses-eye state-reflections-lighting conditions-sensor ID*

Examples of image annotations of the proposed dataset:
![](images\eyedataset_sample.png "Sample Examples")

---

### Take Out 10% of the Data
First, I shall experiment on about a tenth of the dataset and find the most promising models. Finally, I shall train the short-listed models on the full dataset and compare their performance. The best model wins ;)

In [2]:
import os


ROOT_DIR = ""
MRLDATASET_PATH = os.path.join(ROOT_DIR, "mrlEyes_2018_01")
DATA_PATH = os.path.join(ROOT_DIR, "data") # 10% of data would be stored here
os.makedirs(DATA_PATH, exist_ok=True) # create data directory

```
import shutil
import random
from tqdm import tqdm


# List all subject folders
# sd for 'sub_directory'
subjectFolders = [sd for sd in os.listdir(MRLDATASET_PATH) if \
    os.path.isdir(os.path.join(MRLDATASET_PATH, sd))]

# Test
print("The first 5 folders in subjectFolders:", subjectFolders[:6], sep="\n")
print("")
print("The length of subjectFolders is:", len(subjectFolders))
print("")

for folder in tqdm(subjectFolders, desc="Processing and copying files"):
    # Create corresponding folder in destination directory
    os.makedirs(os.path.join(DATA_PATH, folder), exist_ok=True)
    
    # List all image files in the current subject folder
    # f for 'file'
    allFiles = [f for f in os.listdir(os.path.join(MRLDATASET_PATH, folder)) \
        if f.endswith('.png')]
    
    # Shuffle and select 10%
    random.shuffle(allFiles)
    subsetFiles = allFiles[:int(0.1 * len(allFiles))]
    
    # Copy the selected files to the destination directory
    for file in tqdm(subsetFiles, desc=f"Copying files for {folder}"):
        shutil.copy(os.path.join(MRLDATASET_PATH, folder, file), 
        os.path.join(DATA_PATH, folder, file))
    # end for
# end for
```

### Segregating Images into Open and Closed Eyes Directories

In [3]:
listAllFolders = [sd for sd in os.listdir(DATA_PATH) if \
                  os.path.isdir(os.path.join(DATA_PATH, sd))]

listAllFolders

['closed', 'opened']

In [4]:
OPENED_EYE_DIR = os.path.join(DATA_PATH, "opened")
os.makedirs(OPENED_EYE_DIR, exist_ok=True)

CLOSED_EYE_DIR = os.path.join(DATA_PATH, "closed")
os.makedirs(CLOSED_EYE_DIR, exist_ok=True)

In [5]:
sampleImgName = "s0001_00008_0_0_1_0_0_01.png"
print("Eye State:", sampleImgName.split("_")[4])
for i, info in enumerate(sampleImgName.split("_")):
    if i == 4:
        print(i, info, "<-- Eye State")
    else:
        print(i, info)
    # end if-else
# end for

Eye State: 1
0 s0001
1 00008
2 0
3 0
4 1 <-- Eye State
5 0
6 0
7 01.png


```
for folder in tqdm(listAllFolders, desc="Moving images to their \
respective label folder"):
    folderPath = os.path.join(DATA_PATH, folder)

    listAllFiles = [f for f in os.listdir(folderPath) if f.endswith(".png")]

    for file in tqdm(listAllFiles, desc=f"Moving images in folder {folder}"):
        # Check the eye status based on the naming convention of the file
        eyeState = file.split("_")[4]
        
        # 0 ==> "closed", 1 ==> "opened"
        if eyeState == '0':
            shutil.move(os.path.join(folderPath, file), \
                        os.path.join(CLOSED_EYE_DIR, file))
        elif eyeState == '1':
            shutil.move(os.path.join(folderPath, file), \
                        os.path.join(OPENED_EYE_DIR, file))
        # end if-elif
    # end for

    # After moving all images delete the empty folder
    os.rmdir(folderPath)
    print(f"Deleted folder {folder}!")
# end for
```

In [6]:
listAllClosedFiles = [f for f in os.listdir(CLOSED_EYE_DIR) if \
                      f.endswith(".png")]
print("# of closed images:", len(listAllClosedFiles))

listAllOpenedFiles = [f for f in os.listdir(OPENED_EYE_DIR) if \
                      f.endswith(".png")]
print("# of opened images:", len(listAllOpenedFiles))

print("Ratio of closed to opened:", len(listAllClosedFiles) / \
      len(listAllOpenedFiles))

# of closed images: 4187
# of opened images: 4285
Ratio of closed to opened: 0.9771295215869311


### Split the Dataset into Train, Validation, and Test Sets

In [7]:
# Create the directories
TRAIN_DIR = os.path.join(DATA_PATH, 'train')
os.makedirs(TRAIN_DIR, exist_ok=True)

VALID_DIR = os.path.join(DATA_PATH, 'validation')
os.makedirs(VALID_DIR, exist_ok=True)

TEST_DIR = os.path.join(DATA_PATH, 'test')
os.makedirs(TEST_DIR, exist_ok=True)

In [9]:
# Create in each of the above created directories two more directories:
# 'opened' and 'closed'
for directory in (TRAIN_DIR, VALID_DIR, TEST_DIR):
    openedEyeDirHold = os.path.join(directory, 'opened')
    os.makedirs(openedEyeDirHold, exist_ok=True)

    closedEyeDirHold = os.path.join(directory, 'closed')
    os.makedirs(closedEyeDirHold, exist_ok=True)
# end for

In [17]:
# Split the data randomly by shuffling
import shutil
import random


def splitAndMoveFiles(src, trainDest, valDest, testDest, splitRatio=(0.8, 0.1, 
                                                                     0.1)):
    """
    Function to split the images in src into trainDest, valDest, testDest by
    splitRatio
    """
    listFiles = [f for f in os.listdir(src) if f.endswith('.png')]
    random.shuffle(listFiles)

    trainPercent = int(splitRatio[0] * len(listFiles))
    valPercent = int(splitRatio[1] * len(listFiles))
    
    trainFiles = listFiles[: trainPercent]
    valFiles = listFiles[trainPercent: (trainPercent + valPercent)]
    testFiles = listFiles[(trainPercent + valPercent): ]

    for files in (trainFiles, valFiles, testFiles):
        destPath = {
            tuple(trainFiles): trainDest,
            tuple(valFiles): valDest,
            tuple(testFiles): testDest,
        }

        for file in files:
            shutil.move(
                os.path.join(src, file),
                os.path.join(destPath[tuple(files)], file)
            )
        # end for
    # end for
    
    return
# end function splitAndMoveFiles()

```
# Function calls
trainDest = os.path.join(TRAIN_DIR, 'opened')
valDest = os.path.join(VALID_DIR, 'opened')
testDest = os.path.join(TEST_DIR, 'opened')

splitAndMoveFiles(OPENED_EYE_DIR, trainDest, valDest, testDest)

trainDest = os.path.join(TRAIN_DIR, 'closed')
valDest = os.path.join(VALID_DIR, 'closed')
testDest = os.path.join(TEST_DIR, 'closed')

splitAndMoveFiles(CLOSED_EYE_DIR, trainDest, valDest, testDest)


# Delete the empty directories
os.rmdir(OPENED_EYE_DIR)
os.rmdir(CLOSED_EYE_DIR)
```

In [24]:
# Check the size of the directories
for directory in (TRAIN_DIR, VALID_DIR, TEST_DIR):
    print(f"In directory -> {directory}")
    subDirs = [sd for sd in os.listdir(directory) \
               if os.path.isdir(os.path.join(directory, sd))]
    
    for subDir in subDirs:
        print(f"    In sub directory -> {subDir}")
        listFiles = [f for f in os.listdir(os.path.join(directory, subDir))
                     if f.endswith('.png')]
        print(f"        # of files: {len(listFiles)}")
    # end for
# end for

In directory -> data\train
    In sub directory -> closed
        # of files: 3349
    In sub directory -> opened
        # of files: 3428
In directory -> data\validation
    In sub directory -> closed
        # of files: 418
    In sub directory -> opened
        # of files: 428
In directory -> data\test
    In sub directory -> closed
        # of files: 420
    In sub directory -> opened
        # of files: 429


### Exploratory Data Analysis (EDA)